In [1]:
# for get the pdf files or url
import requests
# for tree traversal scraping in webpage
from bs4 import BeautifulSoup
import os
# For getting information about the pdfs
from PyPDF2 import PdfFileReader
from urllib.request import unquote

In [2]:
# website to scrap
url = "http://proceedings.mlr.press/v70/"

# get the url from requests get method
read = requests.get(url)

# full html content
html_content = read.content

# Parse the html content
soup = BeautifulSoup(html_content, "html.parser")


In [3]:
pdf_links = []
for link in soup.findAll('a'):
    pdf_links.append(link.get('href'))
pdf_links = [link for link in pdf_links if ".pdf" in link and '-supp' not in link]
print("some pdf_links", pdf_links[:3])

some pdf_links ['http://proceedings.mlr.press/v70/achab17a/achab17a.pdf', 'http://proceedings.mlr.press/v70/acharya17a/acharya17a.pdf', 'http://proceedings.mlr.press/v70/achiam17a/achiam17a.pdf']


In [4]:
print(len(pdf_links))

434


In [5]:
%%time
pdf_contents = " "
for pdf_url in pdf_links:
    pdf_response = requests.get(pdf_url)
    # extract  PDF file name
    filename = unquote(pdf_response.url).split('/')[-1].replace(' ', '_')
    # write PDF to local file
    with open(filename, 'wb') as f:
        # write PDF to local file
        f.write(pdf_response.content)
    # creating a pdf file object
    pdfFileObj = open(filename, 'rb')
    # creating a pdf reader object
    pdfReader = PdfFileReader(pdfFileObj)
    for page in range(0, pdfReader.numPages):
        pageObj = pdfReader.getPage(page)
        # extracting text from page
        pdf_contents = pdf_contents + " " + pageObj.extractText()
        # closing the pdf file object
    pdfFileObj.close()
    os.remove(filename)

CPU times: total: 8min 2s
Wall time: 10min 28s


In [8]:
with open('scrapped_pdf_text.txt', 'w+', encoding="utf-8") as fh:
    fh.write(pdf_contents)